In [14]:
def load_data():
    df=pd.read_csv('D:/mac归档/研究生小论文和毕设/毕业论文/data/bank customer Churn.csv')
    #随机打乱数据
    np.random.seed(1)
    df=df.sample(frac=1).reset_index(drop=True)
    #训练结果，是否流失
    result_var='Exited'
    #分类型数据，需要预处理
    cat_names=['Gender','Geography']
    #数值型数据，可直接输入模型
    cont_names=['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']
    #标签
    Y=df['Exited'].astype('int')
    #类别变量变为独热编码，总的有4个类别变量
    one_hot=pd.DataFrame(columns=['Female','Male','France','Germany','Spain','HasCrCard_Yes','HasCrCard_No','IsActiveMember_Yes','IsActiveMember_No'],index=[i for i in range(10000)])
    one_hot.loc[df.Gender=='Female','Female']=1
    one_hot.loc[df.Gender=='Male','Female']=0
    one_hot.loc[df.Gender=='Male','Male']=1
    one_hot.loc[df.Gender=='Female','Male']=0
    one_hot.loc[df.Geography=='France','France']=1
    one_hot.loc[df.Geography!='France','France']=0
    one_hot.loc[df.Geography=='Germany','Germany']=1
    one_hot.loc[df.Geography!='Germany','Germany']=0
    one_hot.loc[df.Geography=='Spain','Spain']=1
    one_hot.loc[df.Geography!='Spain','Spain']=0
    one_hot.loc[df.HasCrCard==1,'HasCrCard_Yes']=1
    one_hot.loc[df.HasCrCard==0,'HasCrCard_Yes']=0
    one_hot.loc[df.HasCrCard==1,'HasCrCard_No']=0
    one_hot.loc[df.HasCrCard==0,'HasCrCard_No']=1
    one_hot.loc[df.IsActiveMember==1,'IsActiveMember_Yes']=1
    one_hot.loc[df.IsActiveMember==0,'IsActiveMember_Yes']=0
    one_hot.loc[df.IsActiveMember==1,'IsActiveMember_No']=0
    one_hot.loc[df.IsActiveMember==0,'IsActiveMember_No']=1
    #X为数据集，未区分训练集和测试集，用pandas提前划分；标准化处理非独热编码
    X=pd.concat([df[cont_names],one_hot],axis=1).astype('float')
    
    #归一化
    #X=X.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))#生成器最后一层是Tanh(-1,1)，把数据标准化到-1到1之间
    
    #标准化
    #X=pd.concat([df[cont_names].apply(lambda x: (x - np.mean(x)) / np.std(x)),one_hot],axis=1).astype('float')
    
    #print(X.loc[10,:])
    #print(Y.loc[10])
    return X,Y
    #X_.reset_index(drop=True, inplace=True);Y_test.reset_index(drop=True, inplace=True)


########k折划分############        
def get_k_fold_data(k, i, X, y):  ###此过程主要是步骤（1）
    # 返回第i折交叉验证时所需要的训练和验证数据，分开放，X_train为训练数据，X_valid为验证数据   
    assert k > 1
    fold_size = X.shape[0] // k  # 每份的个数:数据总条数/折数（组数）
    X_train, Y_train = pd.DataFrame(), pd.Series()#要用series
    for j in range(k):
        ##idx 为每组 valid
        X_part = X.iloc[j * fold_size: (j + 1) * fold_size,:]
        
        Y_part=y[j * fold_size: (j + 1) * fold_size]
        if j == i: ###第i折作valid
            X_test, Y_test = X_part, Y_part
        #elif X_train is None:
            #X_train, Y_train = X_part, Y_part
        else:
            X_train = pd.concat([X_train, X_part], axis=0) #dim=0增加行数，竖着连接
            Y_train = pd.concat([Y_train, Y_part], axis=0)
    

    X_train.reset_index(drop=True, inplace=True)
    X_test.reset_index(drop=True, inplace=True)
    Y_train.reset_index(drop=True, inplace=True)
    Y_test.reset_index(drop=True, inplace=True)
    
    return X_train,Y_train, X_test, Y_test



In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score,confusion_matrix,f1_score,accuracy_score

import warnings
warnings.filterwarnings("ignore")

In [45]:
def train_start(model):
    k=5#K折交叉验证
    X,Y=load_data()
    precision,recall,F1,acc,AUC=[],[],[],[],[]
    for i in range(k):
        print('第{}折交叉验证\n'.format(i+1))
        X_train,Y_train, X_test, Y_test = get_k_fold_data(k, i, X, Y) 
        fit=model.fit(X_train,Y_train)
        fit_prediction_train =fit.predict(X_train)
        fit_prediction_test = fit.predict(X_test)
        ACC=accuracy_score(fit_prediction_test, Y_test)
        f1=f1_score(fit_prediction_test, Y_test)
        auc=roc_auc_score(Y_test, fit.predict_proba(X_test)[::,1])
        acc.append(ACC);F1.append(f1);AUC.append(auc)
    print('5折交叉验证各指标均值和标准差:\n')
    print(np.mean(acc),np.std(acc),np.mean(F1),np.std(F1),np.mean(AUC),np.std(AUC))
    

LR= LogisticRegression(solver='liblinear')
train_start(LR)

RF=RandomForestClassifier(n_estimators=100)
train_start(RF)

GB=GradientBoostingClassifier(n_estimators=100)
train_start(GB)



第1折交叉验证

第2折交叉验证

第3折交叉验证

第4折交叉验证

第5折交叉验证

5折交叉验证各指标均值和标准差:

0.7898 0.007004284403135005 0.10251708265399309 0.028164204259429084 0.6753882430687235 0.012732685076069956
第1折交叉验证

第2折交叉验证

第3折交叉验证

第4折交叉验证

第5折交叉验证

5折交叉验证各指标均值和标准差:

0.8618 0.006830812543175256 0.5751992646023332 0.009999683533090109 0.8465728922800462 0.009578194160663786
第1折交叉验证

第2折交叉验证

第3折交叉验证

第4折交叉验证

第5折交叉验证

5折交叉验证各指标均值和标准差:

0.8619999999999999 0.004764451699828626 0.5784523136276819 0.021722837736857573 0.8652505263799621 0.010868746650681715


In [46]:
XGB=XGBClassifier()
train_start(XGB)

第1折交叉验证

[11:32:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
第2折交叉验证

[11:32:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
第3折交叉验证

[11:32:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
第4折交叉验证

[11:32:37] WARNING: C:/Users/Admin

In [41]:
SVM= SVC(gamma='auto')
train_start(SVM)

第1折交叉验证

第2折交叉验证

第3折交叉验证

第4折交叉验证

第5折交叉验证

5折交叉验证各指标均值和标准差:

0.7963 0.006454455825241968 0.0 0.0 0.5 0.0
